# 2.1 - Macrobond web API - Categories Exploration

*Performing coverage checks based on Macrobond's Categories*

This notebook aims to provide examples of how to use Macrobond's web API call methods as well as insights on the key attributes used to display the output in an understandable format.

We will focus here on using the Search method based on a **Category** input.
Our data is arranged as a logical hierarchy of categories to help you find or narrow down related datasets quickly

*Full error handling is omitted for brevity*

***

## Importing packages

In [1]:
from macrobond_financial.common import Credentials
from macrobond_financial.web import WebClient

***

## Authentication

If you have a Macrobond's web API account, enter your *client_id* and *client_secret* below.

In [2]:
credentials = Credentials()

***

## Get the data
Feel free to refer to https://api.macrobondfinancial.com/swagger/index.html to get the comprehensive list of web API endpoints and parameters used.

In the example below, we are using here the Search endpoint with filters on Category `inea` and Region `gb`: 
> **Income & Earnings - United Kingdom**

Feel free to use the notebook **1.1 - Macrobond web API - Metadata Navigation** to pull out a list of all available categories and regions.

***

## Visualising the data
Let's evaluate Macrobond's coverage for Financial Accounts-related time series in the United Kingdom.

In [3]:
with WebClient(credentials.client_id, credentials.client_secret) as api:
    data_frame = api.search.search(
        entity_types="TimeSeries",
        must_have_values={"Region": "gb", "category": "inea"},
    ).data_frame(
        columns=[
            "Name",
            "FullDescription",
            "Region",
            "Frequency",
            "Source",
            "FirstRevisionTimeStamp",
        ]
    )
data_frame.head(10)

,Name,FullDescription,Region,Frequency,Source,FirstRevisionTimeStamp
0,gbinea00811,"United Kingdom, Average Weekly Earnings, Total...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z
1,gbinea01201,"United Kingdom, Average Weekly Earnings, Total...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z
2,gbinea01081,"United Kingdom, Average Weekly Earnings, By Se...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z
3,gbinea01351,"United Kingdom, Average Weekly Earnings, By Se...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z
4,gbinea00961,"United Kingdom, Average Weekly Earnings, Priva...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z
5,gbinea01771,"United Kingdom, Average Weekly Earnings, By Se...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z
6,gbinea01621,"United Kingdom, Average Weekly Earnings, By Se...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z
7,gbinea00871,"United Kingdom, Average Weekly Earnings, Publi...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z
8,gbinea01801,"United Kingdom, Average Weekly Earnings, By Se...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z
9,gbinea01531,"United Kingdom, Average Weekly Earnings, Publi...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z


### We will now focus on the Point-in-Time (PiT) series in this coverage check
Let's isolate the first element of the Region attribute. While most of the time series carry one region only, some can have multiple regions with for instance "gb" and "gb,city_[xxx]".

In [4]:
data_frame["RegionString"] = data_frame["Region"].apply(
    lambda x: ", ".join(map(str, x))
)
data_frame.head(10)

,Name,FullDescription,Region,Frequency,Source,FirstRevisionTimeStamp,RegionString
0,gbinea00811,"United Kingdom, Average Weekly Earnings, Total...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z,gb
1,gbinea01201,"United Kingdom, Average Weekly Earnings, Total...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z,gb
2,gbinea01081,"United Kingdom, Average Weekly Earnings, By Se...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z,gb
3,gbinea01351,"United Kingdom, Average Weekly Earnings, By Se...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z,gb
4,gbinea00961,"United Kingdom, Average Weekly Earnings, Priva...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z,gb
5,gbinea01771,"United Kingdom, Average Weekly Earnings, By Se...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z,gb
6,gbinea01621,"United Kingdom, Average Weekly Earnings, By Se...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z,gb
7,gbinea00871,"United Kingdom, Average Weekly Earnings, Publi...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z,gb
8,gbinea01801,"United Kingdom, Average Weekly Earnings, By Se...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z,gb
9,gbinea01531,"United Kingdom, Average Weekly Earnings, Publi...",[gb],monthly,src_gbons,2016-02-17T09:30:00Z,gb


### Let's convert the date-time to years only

In [5]:
data_frame["FirstRevisionYear"] = data_frame["FirstRevisionTimeStamp"].str[:4]

### Displaying the new DataFrame
Let's see how our transformations have been applied by isolating on a few columns: `df.iloc[rows,[columns]]`. Note that we are also dropping NaN values in the FirstRevisionDate column: `df.dropna(subset=['FirstRevisionDate'])`

In [6]:
data_frame_final = data_frame.dropna(subset=["FirstRevisionYear"]).iloc[
    0:1000, [0, 1, 6, 3, 4, 7]
]
data_frame_final

,Name,FullDescription,RegionString,Frequency,Source,FirstRevisionYear
0,gbinea00811,"United Kingdom, Average Weekly Earnings, Total...",gb,monthly,src_gbons,2016
1,gbinea01201,"United Kingdom, Average Weekly Earnings, Total...",gb,monthly,src_gbons,2016
2,gbinea01081,"United Kingdom, Average Weekly Earnings, By Se...",gb,monthly,src_gbons,2016
3,gbinea01351,"United Kingdom, Average Weekly Earnings, By Se...",gb,monthly,src_gbons,2016
4,gbinea00961,"United Kingdom, Average Weekly Earnings, Priva...",gb,monthly,src_gbons,2016
...,...,...,...,...,...,...
3255,ons_k58f_emp_a,"United Kingdom, ONS, Average Weekly Earnings, ...",gb,annual,src_gbons,2021
3256,ons_k5dx_emp_m,"United Kingdom, ONS, Average Weekly Earnings, ...",gb,monthly,src_gbons,2020
3257,ons_k58f_emp_m,"United Kingdom, ONS, Average Weekly Earnings, ...",gb,monthly,src_gbons,2020
3258,ons_k58h_emp_m,"United Kingdom, ONS, Average Weekly Earnings, ...",gb,monthly,src_gbons,2020


### Group the results by FirstRevisionYear and Frequency
Note that Macrobond started to systematically collect PiT data in 2018. 
PiT coverage prior to 2018 has been backfilled by leveraging the source or internal collection logs.

In [7]:
df_group = (
    data_frame_final.groupby(["FirstRevisionYear", "Frequency"])["Name"]
    .count()
    .reset_index(name="Count")
)
df_group

,FirstRevisionYear,Frequency,Count
0,2015,monthly,1
1,2016,monthly,73
2,2018,annual,9
3,2018,monthly,4
4,2018,quarterly,5
5,2019,annual,1
6,2019,monthly,2
7,2019,quarterly,1
8,2020,annual,195
9,2020,monthly,392
